# Agent


### 핵심 구성 요소
**1. Model (언어 모델)**

In [1]:
from langchain.agents.middleware import wrap_model_call
from langchain.agents import create_agent
from langchain.chat_models import init_chat_model

# 모델 풀 정의
models = {
    'fast': init_chat_model('gpt-4o-mini'),
    'smart': init_chat_model('gpt-4o')
}

# 모델을 동적으로 선택할 때는 wrap_model_call 미들웨어 사용
@wrap_model_call
def select_model(request, state):
    """복잡도에 따라 모델 동적 선택""" 
    messages = state.get("messages", [])

    if len(messages) > 10 or any("분석" in str(m) for m in messages):
        request.model = models['smart']
    else:
        request.model = models['fast']
    
    return request

agent = create_agent(
    model=models["fast"],  # 기본 모델
    tools=[],
    middleware=[select_model]
)

**2. Tools (도구)**

- 웹 검색으로 실시간 정보 조회

- 코드 실행 및 계산

- 데이터베이스 쿼리

- 외부 API 호출

In [8]:
from langchain.chat_models import init_chat_model
from langchain.agents import create_agent
from langchain.tools import tool
from langchain_community.tools import TavilySearchResults

llm = init_chat_model("gpt-4o-mini")
# 내장 도구
search_tool = TavilySearchResults(max_results=3)

# 커스텀 도구
@tool
def calculate(expression: str) -> str:
    """수학 표현식을 계산합니다."""
    return str(eval(expression))

agent = create_agent(
    model=llm,
    tools=[search_tool, calculate],
    system_prompt="당신은 검색과 계산을 수행하는 어시스턴트입니다."
)

**3. System Prompt (시스템 프롬프트)**

에이전트의 역할과 동작 방식을 정의하는 지침

In [5]:
system_prompt = """
## 역할
- 주식, 채권, 암호화폐 등 금융 정보를 분석합니다.
- 최신 시장 동향을 검색하여 정확한 정보를 제공합니다.

## 작업 방식
1. 사용자 질문을 분석합니다.
2. 필요한 정보를 웹 검색으로 조회합니다.
3. 수치 계산이 필요하면 계산 도구를 사용합니다.
4. 결과를 종합하여 명확하게 답변합니다.

## 주의사항
- 투자 조언은 제공하지 않습니다.
- 출처를 명시합니다.
"""

agent = create_agent(
    model=llm,
    tools=[search_tool, calculate],
    system_prompt=system_prompt
)

In [ ]:
from langchain.agents.middleware import dynamic_prompt
from langchain.agents import create_agent

# 프롬프트를 동적으로 생성할 때는 dynamic_prompt 미들웨에 사용
@dynamic_prompt
def personalized_prompt(request, state):
    """사용자 정보에 따라 프롬프트 동적 생성"""
    user_name = state.get("user_name", "사용자")
    language = state.get("language", "ko")

    if language == "ko":
        return f"""안녕하세요 {user_name}님, 저는 AI 어시스턴트입니다.update_display
        ## 역할
        - 사용자의 질문에 친절하게 답변합니다.
        - 필요시 도구를 사용하여 정보를 조회합니다.

        ## 주의사항
        - 항상 한국어로 답변합니다.
        - 존댓말을 사용합니다."""
    else:
        return f"Hello {user_name}, I'm your AI assistant."

agent = create_agent(
    model=llm,
    tools=[search_tool],
    middleware=[personalized_prompt]  # 동적 프롬프트 미들웨어
)

### 에이전트 실행

In [9]:
# 스트리밍 실행
for event in agent.stream(
    {"messages": [{"role": "user", "content": "서울 날씨 알려줘"}]},
    stream_mode="updates"
):
    print(event)

{'model': {'messages': [AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 20, 'prompt_tokens': 120, 'total_tokens': 140, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_29330a9688', 'id': 'chatcmpl-D19Ulth6pQcoHiY3m0RibOHxvVPxM', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019bea94-aab8-71b2-8a23-e95b42294828-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': '서울 날씨'}, 'id': 'call_dwmlpj6o8a2RZXRnyhzzH8H7', 'type': 'tool_call'}], invalid_tool_calls=[], usage_metadata={'input_tokens': 120, 'output_tokens': 20, 'total_tokens': 140, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'

In [10]:
# 스트리밍 실행
for event in agent.stream(
    {"messages": [{"role": "user", "content": "서울 날씨 알려줘"}]},
    stream_mode="messages"
):
    print(event)

(AIMessageChunk(content='', additional_kwargs={}, response_metadata={'model_provider': 'openai'}, id='lc_run--019bea98-0916-7ce1-a392-333f848eeb9b', tool_calls=[{'name': 'tavily_search_results_json', 'args': {}, 'id': 'call_dJAM1OedchU1ouvKZFFvRaBh', 'type': 'tool_call'}], invalid_tool_calls=[], tool_call_chunks=[{'name': 'tavily_search_results_json', 'args': '', 'id': 'call_dJAM1OedchU1ouvKZFFvRaBh', 'index': 0, 'type': 'tool_call_chunk'}]), {'langgraph_step': 1, 'langgraph_node': 'model', 'langgraph_triggers': ('branch:to:model',), 'langgraph_path': ('__pregel_pull', 'model'), 'langgraph_checkpoint_ns': 'model:7f15738e-7019-0ba4-cff5-c955f3ab2fda', 'checkpoint_ns': 'model:7f15738e-7019-0ba4-cff5-c955f3ab2fda', 'ls_provider': 'openai', 'ls_model_name': 'gpt-4o-mini', 'ls_model_type': 'chat', 'ls_temperature': None})
(AIMessageChunk(content='', additional_kwargs={}, response_metadata={'model_provider': 'openai'}, id='lc_run--019bea98-0916-7ce1-a392-333f848eeb9b', tool_calls=[{'name': '

### 대화 히스토리 관리
여러 턴의 대화를 처리하려면 메시지 히스토리를 유지합니다.

In [11]:
from langchain_core.messages import HumanMessage, AIMessage

messages = []

# 첫번째 대화
messages.append(HumanMessage(content="서울 날씨 알려줘"))
result = agent.invoke({"messages": messages})

# 응답을 히스토리에 추가
messages.extend(result["messages"])

# 후속 질문 (컨텍스트 유지)
messages.append(HumanMessage(content="그럼 우산 필요할까?"))
result = agent.invoke({"messages": messages})

print(result["messages"][-1].content)

내일 서울의 날씨 예보에 따르면 흐리고 한때 비가 올 것으로 예상되며, 강수 확률이 60%입니다. 따라서 우산을 가져가는 것이 좋습니다.


In [12]:
result

{'messages': [HumanMessage(content='서울 날씨 알려줘', additional_kwargs={}, response_metadata={}, id='4deed19e-8dac-4550-b178-bc2ce4ccdfcf'),
  AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 20, 'prompt_tokens': 120, 'total_tokens': 140, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_29330a9688', 'id': 'chatcmpl-D19at8Jk9DrBZD5Gsb2wLbQsWq3Yd', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019bea9a-775a-70c1-9a70-0abf50f02768-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': '서울 날씨'}, 'id': 'call_bojPtbg4K8Md2N5rDDE6sLW6', 'type': 'tool_call'}], invalid_tool_calls=[], usage_metadata={'input_tokens': 120, 'output_

In [15]:
from langchain.agents import create_agent
from langchain.tools import tool
from langchain_community.tools import TavilySearchResults
from langchain_openai import ChatOpenAI

search_tool = TavilySearchResults(max_results=3)

@tool
def calculate(expression: str) -> str:
    """수학 표현식을 계산합니다.

    Args:
        expression: 계산할 수학 표현식 (예: "100 * 1.25")
    """

    try:
        return str(eval(expression))
    except Exception as e:
        return f"계산 오류: {e}"

llm = ChatOpenAI(model="gpt-4o", temperature=0)

agent = create_agent(
    model=llm,
    tools=[search_tool, calculate],
    system_prompt="""당신은 리서치와 분석 전문 어시스턴트입니다.
    ## 역할
    - 웹 검색으로 최신 정보를 조회합니다.
    - 수치 계산이 필요하면 calculate 도구를 사용합니다.
    - 결과를 종합하여 명확하게 답변합니다.
    
    ## 답변 형식
    - 핵심 내용을 먼저 요약합니다.
    - 세부 정보를 bullet point로 정리합니다.
    - 출처를 마지막에 포함합니다.
    """
)

result = agent.invoke({
    "messages": [{
        "role": "user",
        "content": "오늘 애플 주가를 검색하고, 작년 대비 수익률을 계산해줘"
    }]
})

print(result["messages"][-1].content)

오늘 애플 주가는 약 $248.01이며, 작년 같은 시점의 주가는 약 $271.81였습니다. 따라서 작년 대비 수익률은 약 -8.76%입니다.

- **오늘의 주가**: $248.01
- **작년의 주가**: $271.81
- **수익률**: -8.76%

출처:
- [Robinhood](https://robinhood.com/us/en/stocks/AAPL/)
- [Trading Economics](https://tradingeconomics.com/aapl:us)
